In [1]:
import sys
sys.path.append('../')
import numpy as np
from utility.file_utility import FileUtility
import random
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import TfidfVectorizer
from utility.math_utility import normalize_mat

In [2]:
ez_fasta_dict=FileUtility.read_fasta_sequences_ids('/mounts/data/proj/asgari/dissertation/git_repos/16S_datasets/EZ/raw/eztaxon_qiime_full.fasta')
ez_taxa_dict={x.split()[0]:x.split()[1].split(';') for x in FileUtility.load_list('/mounts/data/proj/asgari/dissertation/git_repos/16S_datasets/EZ/raw/eztaxon_id_taxonomy.txt')}
ez_seq_keys=list(ez_fasta_dict.keys())
ez_seq_keys.sort()
ez_taxa_keys=list(ez_taxa_dict.keys())
ez_taxa_keys.sort()
ez_taxa_keys==ez_seq_keys

True

In [3]:
from make_representations.cpe_apply import CPE
import tqdm
from multiprocessing import Pool

In [4]:
f=open('../../16S_datasets/ra/cpe/ra_cpe','r')
CPE_Applier=CPE(f, merge_size=-1,separator='')

In [5]:
segmentation_input=[(x,y[0]) for x,y in ez_fasta_dict.items()]

In [6]:
ez_segmented=dict()
pool = Pool(processes=20)
for key, v in tqdm.tqdm(pool.imap_unordered(CPE_Applier.segment_with_keys, segmentation_input, chunksize=5),
                       total=len(segmentation_input)):
    ez_segmented[key]=v

100%|██████████| 63240/63240 [18:43<00:00, 56.30it/s]


In [17]:
out_dir='/mounts/data/proj/asgari/dissertation/git_repos/16S_datasets/ra/specific_ez/'
# save the segmented corpus
FileUtility.save_obj(out_dir+'segmented_ez',ez_segmented)

100%|██████████| 63240/63240 [00:00<00:00, 406984.05it/s]


In [16]:
x,y

('127466',
 'acaca tgcaag tcg agggg cagca tggtc ttag cttg ctaagg acg atgg cg accgg cgca cggg tgag taa cacg tatcc aacctg ccgtcta ctc ttggac agcc ttctg aaagg aag attaata caag atgg catcg tgag tccg catg ttc tcatg atta aaggta ttccgg tagacg atggg catg cgttcc atta catag tagg cgagg taacgg ccca cctag tcttcc atgga taggg gta ctgag aggaagg tcccc caca ttgg aactg aga cacg gacc aaac tccta cggg aggcag cag tgaag aata ttgg tcaa tggg cgca agcctg aaccag cca agtag cgtgaagg atg actgcc ctaagg attg taaa cttctt ttata cagg aata acg tcggata tg aatacc cgtttg ca tgta cttta tg aataagg atcggcta actccg tg tcagcag ccg cggta ata ccgagg atccg agtg tta tccgga ttta ttggg tttaaaggg agcg tag atgga tg tttaag tcag ttgtg aaactttg ccg ctcaa ccgta aaattg cagttg ata ctgga ta tcttg agtg cagttg aggcagg cggaa ttcg tgg tgta tcgg tgaaatg cttag atatca cgaag aa ctccga ttgcg aaggc agcctg cta agctg caactg aca ttgagg ctcg aaag tgtggg ta tcaaacagg attag atacc ctgg tag tcca cacggta aacg atgaa tag tagctg tgtgcg ata tacgg caag cgg ccaag cgatag

In [19]:
levels={'phylum':1,'class':2,'order':3,'family':4,'genus':5,'species':6}
seq_taxa=ez_taxa_dict

In [ ]:
for level in levels:
    print (level)
    corpus=dict()
    print ('corpus reading')
    for seq_idx,seq in tqdm.tqdm(ez_segmented.items()):
        if seq_taxa[seq_idx][levels[level]] not in corpus:
            corpus[seq_taxa[seq_idx][levels[level]]]=''
        corpus[seq_taxa[seq_idx][levels[level]]]+=' '+seq
    print ('tf-idf assignments')
    tax_ids=list(corpus.keys())
    tax_ids.sort()
    texts=[corpus[idx] for idx in tax_ids]
    tf_vectorizer = TfidfVectorizer(use_idf=False, analyzer='word',
                                            norm=None, stop_words=[], lowercase=True, binary=False, smooth_idf=False, tokenizer=str.split)
    tf_vec = tf_vectorizer.fit_transform(texts)
    feature_names = tf_vectorizer.get_feature_names()
    print ('normalization')
    normalize_matrix=normalize_mat(tf_vec,axis=0)
    normalize_matrix=normalize_matrix.toarray()
    prob=[' '.join(['###'.join([tax_ids[x],str(np.round(normalize_matrix[x,i],5))]) for x in np.nonzero(normalize_matrix[:,i])[0].tolist() if np.round(normalize_matrix[x,i],5) > 0.1]) for i in range(normalize_matrix.shape[1])]
    print ('saving')
    FileUtility.save_sparse_csr(out_dir+'ez_tf_'+level+'_mat', tf_vec)
    FileUtility.save_list(out_dir+'ez_'+level,tax_ids)
    FileUtility.save_list(out_dir+'ez_cpe_'+level+'_prob',prob)
    FileUtility.save_list(out_dir+'ez_cpe_'+level+'_features',feature_names)

  2%|▏         | 1454/63240 [00:00<00:04, 14524.26it/s]

family
corpus reading


 42%|████▏     | 26349/63240 [03:05<07:50, 78.38it/s]  